In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from itertools import product

In [ ]:
# notice the changes compared to commons.smoothen
def smoothen(xs, window):
    l = xs.shape[-1]
    return np.moveaxis([
        xs[..., max(0, t-window) : min(l, t+window)]
            .mean(axis = len(xs.shape)-1)
        for t in range(l)
    ], 0, 3)

# Preparations

## Loading the data

We first load the $N_i(t)$, while smoothening them (window = 5) :

In [ ]:
Nt = smoothen(np.load("curves_raw.npy"), 5)

Let's also compute the discrete derivatives $\frac{\Delta N_i(t)}{\Delta t} = \Delta N_i(t) = N_i(t+1) - N_i(t)$

In [ ]:
dNdt = Nt[..., 1:] - Nt[..., :-1]

and $\rho_i(t) = \frac{\Delta N_i(t)}{N_i(t)}$

In [ ]:
rho = dNdt / Nt[..., :-1]

We now perform similar computations for another set of $\rho_i(t)$ but with more aggressive smoothening, whose purpose is to provide valid times until maximum growth is reached (i.e. argmax happens after t=0) :

In [ ]:
nt = smoothen(np.load("curves_raw.npy"), 10)

In [ ]:
dndt = nt[..., 1:] - nt[..., :-1]

In [ ]:
dnn = smoothen(dndt / nt[..., :-1], 6)

## Dimensions

Let's also get the dimensionality of our data :

In [ ]:
n_plates, n_rows, n_columns, n_points = Nt.shape
plates, rows, columns, points = map(np.arange, Nt.shape)

# Model-aware $\alpha_i(t)$

Let's assume the model
$$ \alpha_i(t) = r_0 \frac{c_i}{c_i + e^{-m t}} $$
It has the variables
* $r_0$ the *maximum growth rate*
* $c_i$ the *internal physiological state*
* $m$ the *adjustment rate to growth conditions*

## Population-specific $r_{0,i}$ and $c_i$ and $m_i$

Let's first find local ($r_{0,i}$, $m_i$ and $c_i$) to fit $\rho_i(t) = \alpha_i(t)$ :

In [ ]:
params0 = pd.DataFrame(
    data    = np.empty((n_plates * n_rows * n_columns, 3)),
    columns = ("r0 i", "m i", "c i"),
    index   = pd.MultiIndex.from_product((plates, rows, columns), names = ("plate", "row", "column"))
)
params0["r0 i"] = rho.max(axis = 3).reshape(-1)

In [ ]:
for idx in product(plates, rows, columns):
    t_max = dnn[idx].argmax()
    
    params0.loc[idx] = curve_fit(
        f      = lambda t, r_0i, m_i, c_i:
                    r_0i * c_i / (c_i + np.exp(-m_i * t)),
        xdata  = np.arange(t_max),
        ydata  = rho[idx][:t_max],
        p0     = (rho[idx].max(), 0.1, 1),
        bounds = (0, (2*rho[idx].max(), 5, 10))
    )[0]

Let's first find the $(c_i, m_i)$ to fit $\rho_i(t) = \alpha_i(t)$, calculating $r_{0,i}$ from the maximum of each curve :

In [ ]:
for idx in product(plates, rows, columns):
    r0_i, t_max = params0.loc[idx, "r0 i"], dnn[idx].argmax()
    
    params0.loc[idx][1:] = curve_fit(
        f      = lambda t, m_i, c_i:
                    r0_i * c_i / (c_i + np.exp(-m_i * t)),
        xdata  = np.arange(t_max),
        ydata  = rho[idx][:t_max],
        p0     = (0.1, 1),
        bounds = (0, np.inf)
    )[0]

In [ ]:
params0.to_csv("alpha/params0.csv")

## Global values for $r_0$ and $m$

First let's make the input/output values consistent with the individual growth to the maximum by padding with a deterministic value :

In [ ]:
ts  = np.full((n_plates, n_rows, n_columns, n_points), np.nan)
cis = np.full((n_plates, n_rows, n_columns, n_points), np.nan)
ys  = np.full((n_plates, n_rows, n_columns, n_points), np.nan)

for idx in product(plates, rows, columns):
    t_max = dnn[idx].argmax()
    
    ts[idx][:t_max]  = np.arange(t_max)
    cis[idx][:t_max] = params0.loc[idx, "c i"]
    ys[idx][:t_max]  = rho[idx][:t_max]

Let's now find the global values for $r_0$ and $m$ :

In [ ]:
r0_m = pd.DataFrame(
    data    = np.empty((n_plates, 2)),
    columns = ("r0", "m"),
    index   = pd.Index(plates, name = "plate")
)

In [ ]:
for p in plates:
    ci = pd.Series(cis[p].reshape(-1)).dropna()
    
    r0_m.loc[p] = curve_fit(
        f     = lambda t, r0, m:
                    r0 * ci / (ci + np.exp(-m * t)),
        xdata = pd.Series(ts[p].reshape(-1)).dropna(),
        ydata = pd.Series(ys[p].reshape(-1)).dropna(),
        p0    = tuple(params0.loc[p, ["r0 i", "m i"]].mean())
    )[0]

# Recomputing the $c_i$

Let's finally recompute the $c_i$ :

In [ ]:
c_i = pd.Series(
    data  = np.empty(n_plates * n_rows * n_columns),
    index = params0.index
)

In [ ]:
for p in plates:
    r0, m = r0_m.loc[p]
    
    for r, c in product(rows, columns):
        idx = (p, r, c)
        t_max = dnn[idx].argmax()
        
        c_i[idx] = curve_fit(
            f      = lambda t, ci:
                        r0 * ci / (ci + np.exp(-m * t)),
            xdata  = np.arange(t_max),
            ydata  = rho[idx][:t_max],
            bounds = (0, np.inf)
        )[0]

Let's make a proper data frame for these parameters :

In [ ]:
params = params0.copy()
for p in plates:
    params.loc[p, "r0 i"] = r0_m.loc[p, "r0"]
    params.loc[p, "m i"]  = r0_m.loc[p, "m"]
    params.loc[p, "c i"]  = c_i

In [ ]:
params.to_csv("alpha/params.csv")

# Computing the $\alpha_i(t)$

Let's compute the $\alpha_i(t)$ from these parameters, and store them :

In [ ]:
alpha = np.empty((4, 32, 48, 218))
for idx in product(plates, rows, columns):
    r0, mi, ci = params.loc[idx]
    alpha[idx] = r0 * ci / (ci + np.exp(-mi * np.arange(218)))

In [ ]:
np.save("alpha/computed.npy", alpha)